In [2]:
import json
import os

# --- Configuration ---

# Set the path to your annotations file
FILE_PATH = r'D:\IITGN\Placement_prep\Real-Estate-Search-Engine\assets\annotations.coco.json'

# The category IDs that represent a "room_name"
ROOM_NAME_IDS = {0, 5}

# The new labels you want to add
NEW_LABELS = ['bedroom', 'kitchen', 'bathroom', 'garage', 'hall']

# --- End of Configuration ---


def get_new_category_id(area, mapping):
    """
    Applies the logic rules you provided to find the correct new category ID.
    'mapping' is a dict like {'bedroom': 8, 'kitchen': 9, ...}
    """
    if area > 3000:
        return mapping.get('bedroom')
    elif 1500 < area <= 3000:
        return mapping.get('kitchen')
    elif 1000 < area <= 1500:
        return mapping.get('bathroom')
    elif 500 < area <= 1000:
        return mapping.get('garage')
    elif area <= 500:
        return mapping.get('hall')
    
    # Return None if no rule matches (shouldn't happen with these rules)
    return None

def process_annotations(file_path):
    """
    Reads, processes, and overwrites the annotations file.
    """
    
    # --- 1. Safety Check ---
    if not os.path.exists(file_path):
        print(f"Error: File not found at '{file_path}'")
        return

    # --- 2. Read the file ---
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        print(f"Successfully read '{file_path}'.")
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from '{file_path}'.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while reading: {e}")
        return

    # --- 3. Add New Categories (if they don't already exist) ---
    existing_cat_names = {cat['name'] for cat in data['categories']}
    new_cats_to_add = [label for label in NEW_LABELS if label not in existing_cat_names]
    
    if new_cats_to_add:
        print(f"Adding new categories: {', '.join(new_cats_to_add)}")
        # Find the highest current ID to start from
        max_id = max(cat['id'] for cat in data['categories'])
        next_id = max_id + 1
        
        for label in new_cats_to_add:
            data['categories'].append({
                "id": next_id,
                "name": label,
                "supercategory": "room_name" # Matches your existing structure
            })
            next_id += 1
    else:
        print("All new categories already exist in the file.")

    # Create a simple mapping of { 'label_name': id } for easy lookup
    # This map will work even if the categories already existed
    label_to_id_map = {cat['name']: cat['id'] for cat in data['categories']}
    
    # Filter this map to only the new labels we care about
    rule_id_map = {label: label_to_id_map.get(label) for label in NEW_LABELS}
    
    # --- 4. Process Annotations and Count Rooms ---
    print("Processing annotations and counting rooms...")
    
    # Initialize a room counter for each image
    image_room_count = {img['id']: 0 for img in data['images']}
    annotations_updated = 0

    for ann in data['annotations']:
        # Check if this annotation is a 'room_name'
        if ann['category_id'] in ROOM_NAME_IDS:
            
            # a) Increment the room count for its parent image
            image_id = ann['image_id']
            if image_id in image_room_count:
                image_room_count[image_id] += 1
            
            # b) Get the area and find the new category ID based on your rules
            area = ann.get('area', 0)
            new_id = get_new_category_id(area, rule_id_map)
            
            # c) Update the annotation's category_id *in-place*
            if new_id is not None:
                ann['category_id'] = new_id
                annotations_updated += 1
            else:
                print(f"Warning: No rule matched for annotation {ann['id']} with area {area}")

    print(f"Updated {annotations_updated} annotation categories based on area.")

    # --- 5. Add 'room_count' to Image Entries ---
    for img in data['images']:
        img['room_count'] = image_room_count.get(img['id'], 0)
    
    print("Added 'room_count' to all image entries.")

    # --- 6. Write Back to the Original File ---
    try:
        with open(file_path, 'w') as f:
            # indent=4 makes the file human-readable
            json.dump(data, f, indent=4)
        print(f"Successfully updated and overwrote '{file_path}'.")
    except Exception as e:
        print(f"Error writing changes back to file: {e}")

# --- Run the script ---
if __name__ == "__main__":
    # Make sure to change 'annotations.json' if your file has a different name
    process_annotations(FILE_PATH)

Successfully read 'D:\IITGN\Placement_prep\Real-Estate-Search-Engine\assets\annotations.coco.json'.
Adding new categories: bedroom, kitchen, bathroom, garage, hall
Processing annotations and counting rooms...
Updated 7737 annotation categories based on area.
Added 'room_count' to all image entries.
Successfully updated and overwrote 'D:\IITGN\Placement_prep\Real-Estate-Search-Engine\assets\annotations.coco.json'.


In [3]:
import json

def assign_room_label(area):
    """
    Assign room label based on bounding box area.
    
    bedroom: area > 3000
    kitchen: 1500 < area <= 3000
    bathroom: 1000 < area <= 1500
    garage: 500 < area <= 1000
    hall: area <= 500
    """
    if area > 3000:
        return "bedroom"
    elif area > 1500:
        return "kitchen"
    elif area > 1000:
        return "bathroom"
    elif area > 500:
        return "garage"
    else:
        return "hall"

# Read the annotations file
file_path = r"D:\IITGN\Placement_prep\assets2\assets\train\annotations.coco.json"
with open(file_path, 'r') as f:
    data = json.load(f)

# Process annotations and add room labels
print("Processing annotations...")
print(f"Total annotations: {len(data['annotations'])}\n")

# Track statistics per image
image_stats = {}

for annotation in data['annotations']:
    category_id = annotation['category_id']
    area = annotation['area']
    image_id = annotation['image_id']
    
    # Check if category_id is 0 or 5 (room_name)
    if category_id in [0, 5]:
        # Assign room label based on area
        room_label = assign_room_label(area)
        annotation['room_label'] = room_label
        
        # Track statistics per image
        if image_id not in image_stats:
            image_stats[image_id] = {
                'bedroom': 0,
                'kitchen': 0,
                'bathroom': 0,
                'garage': 0,
                'hall': 0,
                'total_rooms': 0
            }
        
        image_stats[image_id][room_label] += 1
        image_stats[image_id]['total_rooms'] += 1

# Add room statistics to each image
for image in data['images']:
    image_id = image['id']
    if image_id in image_stats:
        image['room_statistics'] = image_stats[image_id]
    else:
        # No rooms detected for this image
        image['room_statistics'] = {
            'bedroom': 0,
            'kitchen': 0,
            'bathroom': 0,
            'garage': 0,
            'hall': 0,
            'total_rooms': 0
        }

# Save the updated data back to the same file
with open(file_path, 'w') as f:
    json.dump(data, f, indent=4)

print("✓ Annotations updated successfully!")
print(f"✓ File saved: {file_path}\n")

# Display statistics
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

total_rooms_all_images = 0
room_type_totals = {'bedroom': 0, 'kitchen': 0, 'bathroom': 0, 'garage': 0, 'hall': 0}

for image_id, stats in image_stats.items():
    print(f"\nImage ID {image_id}:")
    print(f"  Total Rooms: {stats['total_rooms']}")
    print(f"  Bedroom: {stats['bedroom']}")
    print(f"  Kitchen: {stats['kitchen']}")
    print(f"  Bathroom: {stats['bathroom']}")
    print(f"  Garage: {stats['garage']}")
    print(f"  Hall: {stats['hall']}")
    
    total_rooms_all_images += stats['total_rooms']
    for room_type in room_type_totals:
        room_type_totals[room_type] += stats[room_type]

print("\n" + "=" * 60)
print("OVERALL TOTALS")
print("=" * 60)
print(f"Total rooms across all images: {total_rooms_all_images}")
print(f"Bedrooms: {room_type_totals['bedroom']}")
print(f"Kitchens: {room_type_totals['kitchen']}")
print(f"Bathrooms: {room_type_totals['bathroom']}")
print(f"Garages: {room_type_totals['garage']}")
print(f"Halls: {room_type_totals['hall']}")

Processing annotations...
Total annotations: 15872

✓ Annotations updated successfully!
✓ File saved: D:\IITGN\Placement_prep\assets2\assets\train\annotations.coco.json

SUMMARY STATISTICS

Image ID 0:
  Total Rooms: 14
  Bedroom: 2
  Kitchen: 6
  Bathroom: 1
  Garage: 2
  Hall: 3

Image ID 1:
  Total Rooms: 12
  Bedroom: 0
  Kitchen: 1
  Bathroom: 2
  Garage: 8
  Hall: 1

Image ID 2:
  Total Rooms: 16
  Bedroom: 0
  Kitchen: 2
  Bathroom: 6
  Garage: 4
  Hall: 4

Image ID 3:
  Total Rooms: 15
  Bedroom: 0
  Kitchen: 0
  Bathroom: 0
  Garage: 0
  Hall: 15

Image ID 4:
  Total Rooms: 22
  Bedroom: 0
  Kitchen: 0
  Bathroom: 3
  Garage: 12
  Hall: 7

Image ID 5:
  Total Rooms: 21
  Bedroom: 0
  Kitchen: 3
  Bathroom: 2
  Garage: 14
  Hall: 2

Image ID 6:
  Total Rooms: 20
  Bedroom: 1
  Kitchen: 10
  Bathroom: 4
  Garage: 2
  Hall: 3

Image ID 7:
  Total Rooms: 12
  Bedroom: 1
  Kitchen: 2
  Bathroom: 2
  Garage: 7
  Hall: 0

Image ID 8:
  Total Rooms: 9
  Bedroom: 2
  Kitchen: 5
  Bathr